In [ ]:
from PIL import Image, ImageDraw, ImageFont
import random
import json
import cv2
import numpy as np
import os
from IPython.display import Image as ImageDisplay, display
import string
import names

def label_to_json(boxes, image_filename, json_path, img_width, img_height):
    annotations = []
    for (label, xmin, ymin, xmax, ymax) in boxes:
        annotations.append({
            "label": label,
            "coordinates": {
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax
            }
        })
    data = {
        "image": image_filename,
        "annotations": annotations,
        "image_size": {
            "width": img_width,
            "height": img_height
        }
    }
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

def random_size(output_path, boxes):
    img = cv2.imread(output_path)

    width, height = img.shape[1], img.shape[0]
    new_img_width = random.randint(400, 1200)
    resize_factor = new_img_width / width
    new_img_height = int(height * resize_factor)
    
    img = cv2.resize(img, (new_img_width, new_img_height))
    
    zoom = random.uniform(1, 1.5)
    background = (55, 100, 10)
    
    new_h, new_w = int(new_img_height * zoom), int(new_img_width * zoom)
    
    new_img = img
    offset_h, offset_w = 0, 0
    
    corrected_box = []
    for box in boxes:
        xmin = int(box[1] * resize_factor) + offset_w
        ymin = int(box[2] * resize_factor) + offset_h
        xmax = int(box[3] * resize_factor) + offset_w
        ymax = int(box[4] * resize_factor) + offset_h
        corrected_box.append(tuple([box[0], xmin, ymin, xmax, ymax]))
    
    return new_img, corrected_box

def perspective_warp(image, boxes, max_distortion=0.1):
    height, width = image.shape[:2]
    
    src_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

    p1 = [random.randint(0, int(width * max_distortion)), random.randint(0, int(height * max_distortion))]
    p2 = [random.randint(int(width * (1 - max_distortion)), width), random.randint(0, int(height * max_distortion))]
    p3 = [random.randint(int(width * (1 - max_distortion)), width), random.randint(int(height * (1 - max_distortion)), height)]
    p4 = [random.randint(0, int(width * max_distortion)), random.randint(int(height * (1 - max_distortion)), height)]
    
    dst_points = np.float32([p1, p2, p3, p4])
    
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    warped_image = cv2.warpPerspective(image, perspective_matrix, (width, height))

    corrected_box = []
    for bbox in boxes:
        bbox_points = np.float32([[[bbox[1], bbox[2]]], [[bbox[3], bbox[2]]], [[bbox[3], bbox[4]]], [[bbox[1], bbox[4]]]])
        warped_points = cv2.perspectiveTransform(bbox_points, perspective_matrix)

        warped_bbox = np.squeeze(warped_points).tolist()
        warped_bbox = [int(x) for point in warped_bbox for x in point] 
        corrected_box.append(tuple([bbox[0], warped_bbox[0], min(warped_bbox[1], warped_bbox[3]),
                                    warped_bbox[4], max(warped_bbox[5], warped_bbox[7])]))
    
    return warped_image, corrected_box

def add_random_noise(image):
    noise = np.random.randint(0, 50, (image.shape[0], image.shape[1], 3), dtype='uint8')
    noisy_image = cv2.add(image, noise)
    return noisy_image

def adjust_brightness(image):
    brightness = random.uniform(0.5, 1.5)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 1] = hsv[:, :, 1] * brightness
    hsv[:, :, 1][hsv[:, :, 1] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    img_bright = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img_bright

def generate_random_data_india():
    first_name = names.get_first_name()
    last_name = names.get_last_name()
    name = f"{first_name} {last_name}"
    aadhaar_number = ' '.join(''.join(random.choices('0123456789', k=12))[i:i+4] for i in range(0, 12, 4))
    dob = f"{random.randint(1, 28):02}/{random.randint(1, 12):02}/{random.randint(1960, 2006)}"
    gender = random.choice(["Male", "Female"])
    issue_date = f"{random.randint(1, 28):02}/{random.randint(1, 12):02}/2023"
    
    return name, aadhaar_number, dob, gender, issue_date

def create_dummy_aadhaar(template_path, output_path):
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Load template image
    template = Image.open(template_path)
    draw = ImageDraw.Draw(template)

    # Define font paths and sizes
    font_path_times = "/path/to/times.ttf"  # Path to Times New Roman font file
    font_path_times_bold = "/path/to/timesbd.ttf"  # Path to Times New Roman Bold font file
    font_path_calibri = "/path/to/calibri.ttf"  # Path to Calibri font
    font_path_calibri_bold = "/path/to/calibribd.ttf"  # Path to Calibri Bold font

    font_times = ImageFont.truetype(font_path_times, size=20)
    font_times1 = ImageFont.truetype(font_path_calibri, size=11)
    font_times_bold = ImageFont.truetype(font_path_calibri, size=33)

    name, aadhaar_number, dob, gender, issue_date = generate_random_data_india()

    # Define text positions
    text_positions = {
        "Name": ((205, 124), name, font_times),
        "DOB": ((358, 150), dob, font_times),
        "Gender": ((266, 174), gender, font_times),
        "AadhaarNumber": ((211, 279), aadhaar_number, font_times_bold)
    }
    boxes = []
    for key, (position, text, font) in text_positions.items():
        draw.text((position[0], position[1] - 5), text, fill="black", font=font)
        bbox = draw.textbbox((position[0], position[1] - 5), text, font=font)
        boxes.append(tuple([key]) + bbox)

    # Draw and rotate the Issue Date
    issue_date_position = (5, 0)  # Adjust this to place the issue date appropriately
    issue_date_text = f"{issue_date}"
    rotated_text_image = Image.new('RGBA', (200, 100), (255, 255, 255, 0))
    draw_rotated = ImageDraw.Draw(rotated_text_image)
    draw_rotated.text((10, 10), issue_date_text, font=font_times1, fill="black")
    rotated_text_image = rotated_text_image.rotate(90, expand=1)
    
    # Position the rotated text on the template
    template.paste(rotated_text_image, issue_date_position, rotated_text_image)
    bbox = draw_rotated.textbbox((10, 10), issue_date_text, font=font_times1)
    rotated_bbox = (issue_date_position[0], issue_date_position[1],
                    issue_date_position[0] + bbox[3], issue_date_position[1] + bbox[2])
    boxes.append(("IssueDate", *rotated_bbox))

    # Convert template to RGB mode before saving
    template = template.convert("RGB")

    # Save the image
    image_path = output_path + ".jpg"
    json_path = output_path + ".json"
    
    template.save(image_path)

    # Resize and possibly warp the image
    image, corrected_box = random_size(image_path, boxes)
    if random.randint(0, 1) == 1:
        image, corrected_box = perspective_warp(image, corrected_box, max_distortion=0.3)
        
    # Apply additional augmentations
    if random.randint(0, 1) == 1:
        image = add_random_noise(image)
    if random.randint(0, 1) == 1:
        image = adjust_brightness(image)


    # Save final image and JSON
    label_to_json(corrected_box, os.path.basename(image_path), json_path, image.shape[1], image.shape[0])
    cv2.imwrite(image_path, image)
    return image_path

def generate_images(num):
    base_output_path = "synthetic_data/train/india4/"
    os.makedirs(base_output_path, exist_ok=True)
    
    for i in range(num):
        output_path = os.path.join(base_output_path, f"india2_{i}")
        template_path = "aaddhaar_female_masked.jpg"
        file = create_dummy_aadhaar(template_path, output_path)
        display(ImageDisplay(filename=file))

generate_images(500)

In [ ]:
import shutil

def zip_directory(directory, zip_filename):
    # Compress the directory into a zip file
    shutil.make_archive(zip_filename, 'zip', directory)

# Specify the directory to compress and the desired zip file name
directory_to_compress = "synthetic_data/train/india4/"
zip_file_name = "india2"

# Call the function to compress the directory
zip_directory(directory_to_compress, zip_file_name)